In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from copy import copy
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, results as rs,scsdata

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

"""path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'
report_start = pd.to_datetime('2021-20-01 06:00:00',dayfirst=True)
report_end = pd.to_datetime('2021-20-01 18:00:00',dayfirst=True)

at = pd.read_csv(path+'at_prod.csv')
av = pd.read_csv(path+'av_prod.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
fa = pd.read_csv(path + 'fa_dec_jan.csv')

# Create sc object from Class
sc = scsdata.ScsData('scs', av, at, fa)
# Call pre-process methods
sc.pre_process_at()
sc.pre_process_av()
sc.pre_process_faults()

output, R2, num_assets = rs.run_single_model(sc_data=sc, 
            report_start=report_start, report_end=report_end, shift=0, weights=[1], speed=470, 
            picker_present=0.91, availability=0.71)"""

"path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'\nreport_start = pd.to_datetime('2021-20-01 06:00:00',dayfirst=True)\nreport_end = pd.to_datetime('2021-20-01 18:00:00',dayfirst=True)\n\nat = pd.read_csv(path+'at_prod.csv')\nav = pd.read_csv(path+'av_prod.csv')\n#fa = pd.read_csv(path + 'scs_tote_matched.csv')\nfa = pd.read_csv(path + 'fa_dec_jan.csv')\n\n# Create sc object from Class\nsc = scsdata.ScsData('scs', av, at, fa)\n# Call pre-process methods\nsc.pre_process_at()\nsc.pre_process_av()\nsc.pre_process_faults()\n\noutput, R2, num_assets = rs.run_single_model(sc_data=sc, \n            report_start=report_start, report_end=report_end, shift=0, weights=[1], speed=470, \n            picker_present=0.91, availability=0.71)"

In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'at_prod.csv')
av = pd.read_csv(path+'av_prod.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'fa_dec_jan.csv')

In [4]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av"                 (id=1810011593600) was called 1 times


In [5]:
at = feat.pre_process_at(at)

Running: Function "pre_process_at"                 (id=1810011591872) was called 1 times


In [6]:
fa,unmapped = feat.pre_process_fa(scs_raw)

#fa= fa[fa['timestamp']<(fa['timestamp'].max()-pd.to_timedelta(7, unit='D'))]

Running: Function "pre_process_fa"                 (id=1810011832672) was called 1 times
Running: Function "add_code"                 (id=1810011594320) was called 1 times
Running: Function "add_tote_colour"                 (id=1810011832384) was called 1 times
Running: Function "load_tote_lookup"                 (id=1810011594176) was called 1 times
Running: Function "load_module_lookup"                 (id=1810008620768) was called 1 times
Running: Function "load_ID_lookup"                 (id=1810011594608) was called 1 times


In [9]:
fa[fa['Area']=='PLC External']

,Alert ID,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,PLCN,Asset Code,Area,Tote Colour,Loop,Suffix,Pick Station,Quadrant,MODULE,Alert Type
260099,224265.0,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:00,C08,Z,2.0,3436,8.0,C08SCSP07+S13 emergency stop button actuated f...,PLC External,Both,NaN,NaN,False,2.0,C08 External,Fault
260080,71894.0,101830.0,C12: C12+P03 emergency stop button actuated fault,2021-01-08 11:56:38,C12,Z,2.0,3436,12.0,C12+P03 emergency stop button actuated fault,PLC External,Both,NaN,NaN,False,3.0,C12 External,Fault
331631,59951.0,109905.0,C13: C13+002BV51 circuit breaker monitoring 24VDC,2021-01-10 16:38:00,C13,Z,2.0,3222,13.0,C13+002BV51 circuit breaker monitoring 24VDC,PLC External,Both,NaN,NaN,False,4.0,C13 External,Fault
331643,59980.0,110353.0,C13: C13+094BV65 circuit breaker monitoring 24VDC,2021-01-10 16:37:59,C13,Z,2.0,3222,13.0,C13+094BV65 circuit breaker monitoring 24VDC,PLC External,Both,NaN,NaN,False,4.0,C13 External,Fault
331646,270871.0,54161.0,C06: C06+040BV58 circuit breaker monitoring 24VDC,2020-12-02 11:01:00,C06,Z,2.0,3222,6.0,C06+040BV58 circuit breaker monitoring 24VDC,PLC External,Both,NaN,NaN,False,1.0,C06 External,Fault
260097,224254.0,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:10:00,C08,Z,7.0,3436,8.0,C08SCSP07+S13 emergency stop button actuated f...,PLC External,Both,NaN,NaN,False,2.0,C08 External,Fault
259795,25236.0,53672.0,C06: C06 emergency stop monitoring area 07,2021-01-18 01:55:17,C06,Z,9.0,3211,6.0,C06 emergency stop monitoring area 07,PLC External,Both,NaN,NaN,False,1.0,C06 External,Fault
260022,315316.0,117711.0,C14: C14 emergency stop monitoring area 14,2020-12-09 05:19:00,C14,Z,14.0,3211,14.0,C14 emergency stop monitoring area 14,PLC External,Both,NaN,NaN,False,4.0,C14 External,Fault
260023,315317.0,109647.0,C13: C13 emergency stop monitoring area 14,2020-12-09 05:19:00,C13,Z,15.0,3211,13.0,C13 emergency stop monitoring area 14,PLC External,Both,NaN,NaN,False,4.0,C13 External,Fault
259899,203114.0,93643.0,C11: C11 emergency stop monitoring area 10,2020-11-24 16:26:00,C11,Z,17.0,3211,11.0,C11 emergency stop monitoring area 10,PLC External,Both,NaN,NaN,False,3.0,C11 External,Fault


In [10]:
unmapped[~unmapped['Asset'].str.contains('C17')]

,Asset,Occurrence
36,C0904STA206,12
54,C1404ACL202,4


In [7]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Running: Function "floor_shift_time_fa"                 (id=2674962424448) was called 1 times


In [8]:
df,fa_PTT = feat.create_PTT_df(fa_floor,at,av,weights = [1])

#df = feat.log_totes(df) 

Running: Function "create_PTT_df"                 (id=2674962426752) was called 1 times
Running: Function "fault_select"                 (id=2674962424736) was called 1 times
Running: Function "load_PTT_lookup"                 (id=2674963233088) was called 1 times
Running: Function "faults_aggregate"                 (id=2674962425024) was called 1 times
Running: Function "av_at_select"                 (id=2674962425312) was called 1 times
Running: Function "aggregate_availability"                 (id=2674962425600) was called 1 times
Running: Function "aggregate_totes"                 (id=2674962425888) was called 1 times
Running: Function "merge_av_fa_at"                 (id=2674962426464) was called 1 times
Running: Function "fault_select"                 (id=2674962424736) was called 2 times
Running: Function "load_PTT_lookup"                 (id=2674963233088) was called 2 times
Running: Function "faults_aggregate"                 (id=2674962425024) was called 2 times
Running: Func

Running: Function "fault_select"                 (id=2674962424736) was called 14 times
Running: Function "load_PTT_lookup"                 (id=2674963233088) was called 14 times
Running: Function "faults_aggregate"                 (id=2674962425024) was called 14 times
Running: Function "av_at_select"                 (id=2674962425312) was called 14 times
Running: Function "aggregate_availability"                 (id=2674962425600) was called 14 times
Running: Function "aggregate_totes"                 (id=2674962425888) was called 14 times
Running: Function "merge_av_fa_at"                 (id=2674962426464) was called 14 times
Running: Function "fault_select"                 (id=2674962424736) was called 15 times
Running: Function "load_PTT_lookup"                 (id=2674963233088) was called 15 times
Running: Function "faults_aggregate"                 (id=2674962425024) was called 15 times
Running: Function "av_at_select"                 (id=2674962425312) was called 15 times
Run

Running: Function "fault_select"                 (id=2674962424736) was called 27 times
Running: Function "load_PTT_lookup"                 (id=2674963233088) was called 27 times
Running: Function "faults_aggregate"                 (id=2674962425024) was called 27 times
Running: Function "av_at_select"                 (id=2674962425312) was called 27 times
Running: Function "aggregate_availability"                 (id=2674962425600) was called 27 times
Running: Function "aggregate_totes"                 (id=2674962425888) was called 27 times
Running: Function "merge_av_fa_at"                 (id=2674962426464) was called 27 times
Running: Function "fault_select"                 (id=2674962424736) was called 28 times
Running: Function "load_PTT_lookup"                 (id=2674963233088) was called 28 times
Running: Function "faults_aggregate"                 (id=2674962425024) was called 28 times
Running: Function "av_at_select"                 (id=2674962425312) was called 28 times
Run

In [9]:
X,y = md.gen_feat_var(df,target = 'Availability', features = ['Faults','Totes'])


Running: Function "gen_feat_var"                 (id=2674960448960) was called 1 times


In [10]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': 42})
r2_oos, coeff, num_assets = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 100)

Running: Function "split"                 (id=2674965088144) was called 1 times
Running: Function "run_OLS"                 (id=2674965148288) was called 1 times
Running: Function "find_features"                 (id=2674965148000) was called 1 times


C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


Running: Function "cross_validate_r2"                 (id=2674965127808) was called 1 times

Cross Validation Scores: 
 
        R2 Scores
1       0.374424
2       0.363739
3       0.349207
4       0.379792
5       0.385761
Mean    0.370585
STD     0.012913


C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


                                 OLS Regression Results                                
Dep. Variable:           Availability   R-squared (uncentered):                   0.917
Model:                            OLS   Adj. R-squared (uncentered):              0.916
Method:                 Least Squares   F-statistic:                              657.8
Date:                Mon, 08 Feb 2021   Prob (F-statistic):                        0.00
Time:                        17:18:31   Log-Likelihood:                          6089.1
No. Observations:               29515   AIC:                                 -1.120e+04
Df Residuals:                   29028   BIC:                                     -7166.
Df Model:                         487                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

In [46]:
fa_PTT = fa_PTT.copy()
coeff = coeff.copy()
# Create empty dataframe to fill
output = pd.DataFrame(columns=['Asset Code', 'Coefficient', 'PTT'])

# For each pick station
for x in fa_PTT.items():
    x = copy(x)
    # Merge coefficients to fault dataframe
    df = coeff[coeff['Asset Code'].isin(x[1])]
    # Add pickstation column
    df.loc[:, 'PTT'] = str(x[0])
    # Join dataframes together for each pick station
    output = pd.concat([output, df], join='outer', ignore_index=True)

output.rename(columns={'Asset Code': 'ID', 'Coefficient': 'COEFFICIENT'},
              inplace=True)
output.sort_values('PTT', inplace=True)

In [48]:
# Create empty dataframe to fill
output = pd.DataFrame(columns=['Asset Code', 'Coefficient', 'PTT'])

# For each pick station
for x in fa_PTT.items():
    x = copy(x)
    # Merge coefficients to fault dataframe
    df = coeff[coeff['Asset Code'].isin(x[1])]
    # Add pickstation column
    df.loc[:, 'PTT'] = str(x[0])
    # Join dataframes together for each pick station
    output = pd.concat([output, df], join='outer', ignore_index=True)

output.rename(columns={'Asset Code': 'ID', 'Coefficient': 'COEFFICIENT'},
              inplace=True)
output.sort_values('PTT', inplace=True)

In [65]:
scs_raw[scs_raw['Duration'].dt.total_seconds()>23*3600]

,Number,Alert,Entry Time,PLC,Desk,Duration,Fault ID
51372,444345,SCS M08: single module 2 access door open back...,2021-01-12 07:01:08.000,SCSM08,SCSM08,0 days 23:33:49,10144
51376,444344,SCS M08: single module 2 access door open fron...,2021-01-12 06:59:42.000,SCSM08,SCSM08,0 days 23:35:14,10143
79447,188345,C23: C2303BCL151 151B54 sensor fault,2021-01-07 07:58:47.000,C23,P03,0 days 23:01:38,3428
79469,418195,SCS112: carousel: inverter error No.:14 [6.2],2021-01-07 07:48:37.000,SCSM11,SCS112,0 days 23:57:02,5050
89052,446023,SCS M11: single module 1 access door open fron...,2021-01-05 14:35:11.000,SCSM11,SCSM11,0 days 23:30:31,11822
189155,141205,C17: C1701RDC251 251B11 occupancy sensor fault,20/11/2020 07:34,C17,P01,0 days 23:24:25,3416
241701,130631,C16: C1602ACL040 040QM11 motor circuit breaker...,26/11/2020 00:08,C16,P02,0 days 23:58:28,3430


In [64]:
num = len(hours[hours==3600*24])
percentage = (num/len(hours))*100
print(num, percentage)

0 0.0


# Plotting

In [ ]:
Output[Output['AREA'] == 'PTT']['ID'].value_counts()

In [ ]:
Total = Output[['ID','SINGLES']].groupby('ID').sum().reset_index()

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(y='SINGLES',x = 'ID',data = Total.sort_values(by='SINGLES',ascending=False).head(10),color = 'r')
plt.ylabel('Lost Singles')
plt.title('SCS Lost Singles Last 12 Hours')

In [ ]:
PTT081 = Output[['Asset Code','PTT081']].groupby('Asset Code').sum().reset_index()

In [ ]:
plt.figure(figsize=(7,5))
sns.barplot(y='PTT081',x = 'Asset Code',data = PTT081.sort_values(by='PTT081',ascending=False).head(5),color = 'r')
plt.ylabel('Lost Singles')
plt.title('PTT081 Lost Singles Last 12 Hours')

In [ ]:
Total_colour = Output[['BLUEGREY','SINGLES']].groupby('BLUEGREY').sum().reset_index()

In [ ]:
plt.figure(figsize=(5,5))
sns.barplot(y='SINGLES',x = 'BLUEGREY',data = Total_colour.sort_values(by='SINGLES',ascending=False),color = 'r')
plt.ylabel('Lost Singles')
plt.title('SCS Lost Singles Last 12 Hours')

In [ ]:
Total_colour

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df, line_kws={"color": "red"})
plt.xlim(0, 5)

In [ ]:
sns.lmplot(x='TOTES', y='Availability', data=df_week, line_kws={"color": "red"})

In [ ]:
ax = sns.regplot(x="TOTES", y="Availability", data=df_week,
x_estimator=np.mean, logx=True, line_kws={"color": "red"})
plt.ylim(0,1)

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df_week, line_kws={"color": "red"})
plt.xlim(0, 5)

In [ ]:
Shift = [0,0,0,10,10,10,20,20,20]
Weights = [[1],[0.7,0.3],[0.7,0.2,0.1],[1],[0.7,0.3],[0.7,0.2,0.1],[1],[0.7,0.3],[0.7,0.2,0.1]]

In [ ]:
Weights[8]

In [ ]:
y

In [ ]:
at

In [ ]:
fa_floor

In [ ]:
df.drop(['Availability','Blue Tote Loss','Grey Tote Loss','timestamp'],axis=1)

In [ ]:
Outputs = {0.5:'Number One',0.4:'Number Two',-1.4:'Number 3'}

In [ ]:
 Output = Outputs[max(k for k, v in Outputs.items())]

In [ ]:
Outputs[0.456]='Numbers 4'

In [ ]:
Outputs

In [ ]:
print('Selected R2:', max(k for k, v in Outputs.items()))

In [ ]:
fa_floor = feat.floor_shift_time_fa(fa, shift=10)

df,fa_PTT = feat.create_PTT_df(fa_floor,at,av,weights=[0.7,0.2,0.1])
df = feat.log_totes(df) 
df_2week = df[df['timestamp']>end_time - pd.to_timedelta(14, unit='D')]
X,y = md.gen_feat_var(df_2week,target = 'Availability', features = ['Totes','Faults'])
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                    'random_state': None})

R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 30)

Output = rs.create_output(fa_PTT,Coeff,end_time,speed = speed, picker_present = picker_present, availability = availability)

In [107]:
True is not False

True

In [59]:
unmapped

,Asset,Occurrence
0,C1701STA004,2483
1,C1703STA191,2097
2,C1701STA012,1070
3,C1701STA104,826
4,C1702STA032,496
...,...,...
83,C1703RDC142,1
84,C1701ACH007,1
85,C1701ACH102,1
86,C1703BCX041,1


In [60]:
fa.idmax()

AttributeError: 'DataFrame' object has no attribute 'idmax'

,Alert ID,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Area,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
258048,140924.0,130631.0,C16: C1602ACL040 040QM11 motor circuit breaker...,2020-12-28 21:48:06,C16,P02,81929.0,3430,C1602ACL040,C16,Blue,NaN,NaN,False,16.0,0.0,NaN,Fault
265041,151857.0,42600.0,C05: C0502RDC038 038AQ1 roller motor unit fault,2020-12-25 04:56:43,C05,P02,82717.0,3429,C0502RDC038,Module Loop Blue,Blue,Module,C0502,False,5.0,1.0,1,Fault
1044,79527.0,188345.0,C23: C2303BCL151 151B54 sensor fault,2021-01-07 07:58:47,C23,P03,82898.0,3428,C2303BCL151,C23,Blue,Outside,C2303,False,23.0,0.0,NaN,Fault
329144,79549.0,418195.0,SCS112: carousel: inverter error No.:14 [6.2],2021-01-07 07:48:37,SCSM11,SCS112,86222.0,5050,SCS112,Carousel,Grey,carousel,SCSM11,False,0.0,3.0,11,Fault
258933,241836.0,130631.0,C16: C1602ACL040 040QM11 motor circuit breaker...,2020-11-26 00:08:00,C16,P02,86308.0,3430,C1602ACL040,C16,Blue,NaN,NaN,False,16.0,0.0,NaN,Fault


In [69]:
from glob import glob
filename = glob(r"C:\Users\Jamie.williams\Desktop\scs\outputs\*.csv")[0]

In [94]:
load = 0;
attempt = 0;
while load=0:
    #THIS NEEDS FIXING
    if filename in Path(r"C:\Users\Jamie.williams\Desktop\scs\outputs").glob("*.csv"):
        for filename in Path(r"C:\Users\Jamie.williams\Desktop\scs\outputs").glob("*.csv"):
            
            output = pd.read_csv(filename)
            run_ID = re.findall('ML_Output_[0-9]+',str(filename))[0].split('_')[-1]
            load = 1
    else:
        attempt=attempt+1
        if attempt<10:
            sys.sleep(300)
        else:
            sys.exit('File Did Not Appear in Outputs Folder')

False


In [119]:
import re
for filename in Path(r"C:\Users\Jamie.williams\Desktop\scs\outputs").glob("*.csv"):
    print(run_ID)
    

99999999


In [100]:
str(filename)

'C:\\Users\\Jamie.williams\\Desktop\\scs\\outputs\\Coefficient File Example.csv'

In [170]:
if list(Path(r"C:\Users\Jamie.williams\Desktop\scs\outputs").glob("ML_Output_*.csv")):
    print('True')
else:
    print('False')

False


In [159]:
Path(r"C:\Users\Jamie.williams\Desktop\scs\outputs").glob("ML_Output_*.csv")[1].exists()
x for filename in Path(r"C:\Users\Jamie.williams\Desktop\scs\outputs").glob("ML_Output_*.csv") if x.exists():
    if filename.exists():
        print(filename)
    else:
        print('Not there son')

SyntaxError: invalid syntax (<ipython-input-159-1a795c489df7>, line 2)

In [165]:
list(Path(r"C:\Users\Jamie.williams\Desktop\scs\outputs").glob("ML_Output_*.csv"))[0].exists()

IndexError: list index out of range

,Alert ID,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Area,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
